In [1]:
import re

import torch
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True)

INFO 10-08 03:57:44 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=1, served_model_name=deepseek-ai/DeepSeek-Prover-V1.5-RL, use_v2_block_manager=False, num_scheduler_steps=

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-08 03:57:52 model_runner.py:1025] Loading model weights took 12.8725 GB
INFO 10-08 03:57:54 gpu_executor.py:122] # GPU blocks: 2895, # CPU blocks: 546
INFO 10-08 03:57:58 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-08 03:57:58 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-08 03:58:34 model_runner.py:1456] Graph capturing finished in 36 secs.


In [36]:
ugly_prompt = '''
Complete the following Lean 4 code with short explanatory comments:
```lean4\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- Determine all values $x$ in the interval $0\\leq x\\leq 2\\pi $ which satisfy the inequality\n$2\\cos x \\leq \\left| \\sqrt{1+\\sin 2x} - \\sqrt{1-\\sin 2x } \\right| \\leq \\sqrt{2}.$-/\ntheorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)\n  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))\n  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :\n  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by\n
'''

complete_prompt = '''

Complete the following Lean 4 code with short explanatory comments:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Determine all values $x$ in the interval $0\leq x\leq 2\pi $ which satisfy the inequality
$2\cos x \leq \left| \sqrt{1+\sin 2x} - \sqrt{1-\sin 2x } \right| \leq \sqrt{2}.$-/
theorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)
  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))
  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :
  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by
'''

In [43]:
print(complete_prompt)



Complete the following Lean 4 code with short explanatory comments:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Determine all values $x$ in the interval $0\leq x\leq 2\pi $ which satisfy the inequality
ight| \leq \sqrt{2}.$-/qrt{1+\sin 2x} - \sqrt{1-\sin 2x } 
theorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)
  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))
  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :
  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by



In [31]:
prompt = r'''Complete the following Lean 4 code with short explanatory comments:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''

In [50]:
def gen(prompt, sampling_params):
    model_outputs = model.generate(
        [prompt],
        sampling_params,
        use_tqdm=False,
    )

    output = model_outputs[0]
    outputs = output.outputs
    # print(outputs[0].text)
    return outputs

In [40]:
options = gen(complete_prompt + '/-')
print("options\n", options)

options
 [CompletionOutput(index=0, text='\nTo determine all values \\( x \\) in the interval \\( 0 \\leq x \\leq 2\\pi \\) which satisfy the inequality\n\\( 2\\cos x \\leq \\left| \\sqrt{1+\\sin 2x} - \\sqrt{1-\\sin 2x } \\right| \\leq \\sqrt{2} \\), we need to analyze the given conditions and derive the bounds for \\( x \\).\n\n1. **Lower Bound**: We need to show that \\( \\frac{\\pi}{4} \\leq x \\).\n2. **Upper Bound**: We need to show that \\( x \\leq \\frac{7\\pi}{4} \\).\n\nGiven the conditions:\n- \\( 2\\cos x \\leq \\left| \\sqrt{1+\\sin 2x} - \\sqrt{1-\\sin 2x } \\right| \\)\n- \\( \\left| \\sqrt{1+\\sin 2x} - \\sqrt{1-\\sin 2x } \\right| \\leq \\sqrt{2} \\)\n\nWe can use these conditions to derive the required bounds for \\( x \\).\n', token_ids=(185, 1898, 6983, 521, 3039, 12837, 1376, 40616, 279, 254, 11501, 12837, 207, 15, 357, 2848, 1376, 357, 2848, 207, 17, 59, 1804, 40616, 588, 12276, 254, 13626, 185, 65694, 207, 17, 59, 6281, 1376, 357, 2848, 357, 1354, 91, 357, 4221, 

In [41]:
print(options[0].text)


To determine all values \( x \) in the interval \( 0 \leq x \leq 2\pi \) which satisfy the inequality
\( 2\cos x \leq \left| \sqrt{1+\sin 2x} - \sqrt{1-\sin 2x } \right| \leq \sqrt{2} \), we need to analyze the given conditions and derive the bounds for \( x \).

1. **Lower Bound**: We need to show that \( \frac{\pi}{4} \leq x \).
2. **Upper Bound**: We need to show that \( x \leq \frac{7\pi}{4} \).

Given the conditions:
- \( 2\cos x \leq \left| \sqrt{1+\sin 2x} - \sqrt{1-\sin 2x } \right| \)
- \( \left| \sqrt{1+\sin 2x} - \sqrt{1-\sin 2x } \right| \leq \sqrt{2} \)

We can use these conditions to derive the required bounds for \( x \).



In [45]:
print(complete_prompt + '  --')



Complete the following Lean 4 code with short explanatory comments:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Determine all values $x$ in the interval $0\leq x\leq 2\pi $ which satisfy the inequality
ight| \leq \sqrt{2}.$-/qrt{1+\sin 2x} - \sqrt{1-\sin 2x } 
theorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)
  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))
  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :
  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by
  --


In [58]:
sampling_params = SamplingParams(
    temperature=1,
    max_tokens=500,
    top_p=0.95,
    n=10,
    stop = ['-/']
)
prompt = complete_prompt + '  --'
options = gen(prompt, sampling_params)
print("prompt:\n", prompt)
print("options:\n", options)
for i in range(len(options)):
    print(f"option {i}:\n")
    print(options[i].text)

prompt:
 

Complete the following Lean 4 code with short explanatory comments:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Determine all values $x$ in the interval $0\leq x\leq 2\pi $ which satisfy the inequality
ight| \leq \sqrt{2}.$-/qrt{1+\sin 2x} - \sqrt{1-\sin 2x } 
theorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)
  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))
  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :
  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by
  --
options:
 [CompletionOutput(index=0, text=' We need to prove two inequalities: π/4 ≤ x and x ≤ 7π/4.\n  constructor\n  -- First, prove π/4 ≤ x.\n  -- Use the given inequality 2 * cos x ≤ |sqrt(1 + sin 2x) - sqrt(1 - sin 2x)|.\n  -- Since cos x is non-negative in the interval [0, π], we can use the fact that cos x ≥ 0.\n  -- By the given inequali

In [56]:
# higher temp
sampling_params = SamplingParams(
    temperature=1.5,
    max_tokens=500,
    top_p=0.95,
    n=10,
    stop = ['\n']
)
options = gen(complete_prompt + '  --', sampling_params)
# print("options:\n", options)
for i in range(len(options)):
    print(f"option {i}:\n")
    print(options[i].text)

option 0:

 We need to prove two inequalities: π/4 ≤ x and x ≤ 7π/4.
option 1:

 We need to show that \( \frac{\pi}{4} \leq x \leq \frac{7\pi}{4} \).
option 2:

 We need to prove two inequalities:
option 3:

 We need to prove that \( \frac{\pi}{4} \leq x \leq \frac{7\pi}{4} \).
option 4:

 We need to show that π/4 ≤ x ≤ 7π/4 given the inequalities.
option 5:

 Show that the cosine of x is non-negative within the given interval.
option 6:

 We start by noting the given conditions and the goal.
option 7:

 We need to prove two parts: lower bound and upper bound for x.
option 8:

 We need to prove that \( \frac{\pi}{4} \leq x \leq \frac{7\pi}{4} \).
option 9:

 We need to prove that x is in the interval [π/4, 7π/4]


In [59]:
sampling_params = SamplingParams(
    temperature=1,
    max_tokens=500,
    top_p=0.95,
    n=10,
    stop = ['-/']
)
prompt = complete_prompt + '  /-'
options = gen(prompt, sampling_params)
print("prompt:\n", prompt)
print("options:\n", options)
for i in range(len(options)):
    print(f"option {i}:\n")
    print(options[i].text)

prompt:
 

Complete the following Lean 4 code with short explanatory comments:
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Determine all values $x$ in the interval $0\leq x\leq 2\pi $ which satisfy the inequality
ight| \leq \sqrt{2}.$-/qrt{1+\sin 2x} - \sqrt{1-\sin 2x } 
theorem imo_1965_p1 (x : ℝ) (h₀ : 0 ≤ x) (h₁ : x ≤ 2 * Real.pi)
  (h₂ : 2 * Real.cos x ≤ abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))))
  (h₃ : abs (Real.sqrt (1 + Real.sin (2 * x)) - Real.sqrt (1 - Real.sin (2 * x))) ≤ Real.sqrt 2) :
  Real.pi / 4 ≤ x ∧ x ≤ 7 * Real.pi / 4 := by
  /-
options:
 [CompletionOutput(index=0, text='\n  To determine all values \\( x \\) in the interval \\( 0 \\leq x \\leq 2\\pi \\) which satisfy the inequality\n  \\[ 2\\cos x \\leq \\left| \\sqrt{1+\\sin 2x} - \\sqrt{1-\\sin 2x } \\right| \\leq \\sqrt{2}, \\]\n  we need to analyze the behavior of the expression \\( \\left| \\sqrt{1+\\sin 2x} - \\sq